In [ ]:
MODEL = "llama2"

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

# model.invoke("Tell me a joke?")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke?")

In [27]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("cv.pdf")

pages = loader.load_and_split()

pages

[Document(metadata={'source': 'cv.pdf', 'page': 0, 'page_label': '1'}, page_content='Sr. Python Engineer | Sr. Full Stack Engineer | Sr. Backend Developer \n \nEmail:    dontay.mobley.7p@outlook.com                 Contact Number: +1717 456 0046 \nLocation: Philadelphia, PA     Work Authorization: US citizen \n \nPROFESSIONAL SUMMARY \n\uf06c Over 10 years of experience in Design, Development and Deployment of Python applications. \n\uf06c Developed backend modules using Python on Django, Flask Web Framework using ORM models. \n\uf06c Developed ETL jobs in Python to extract and load data into various databases. \n\uf06c Used Test driven approach for developing the application and Implemented the unit tests using Python \nUnit test framework.  \n\uf06c Expertise in working with Perl and Shell scripting for designing automation framework. \n\uf06c Good experience in developing web applications and implementing Model View Control (MVC) \narchitecture using server - side applications like 

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know."

Context: {context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="What is your name?")


In [ ]:
chain = prompt | model | parser

In [ ]:
chain.invoke({
    "context": "The name I was given is Ali",
    "question": "What is your name?", 
})

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("Expertise in Machine Learning")

In [ ]:
from operator import itemgetter
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    } |
    prompt |
    model |
    parser
)

chain.invoke({
    "question": "Has he worked on Python?",
})

In [ ]:
itemgetter("question")({"question": "What is your name?"})

In [ ]:
for s in chain.stream({"question": "How many years of experience do the candidate have?"}):
    print(s, end="", flush=True)